## CRISPR repair amplicon assay

The raw fastq files were previously first checked with **Fastq**, trimmed for adapters and low quality regions with **Trimmomatic** and the overlapping forward and reverse reads were merged with **FLASH2**.  

In [ ]:
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import re
import matplotlib.pyplot as plt
import numpy as np
import os
import gzip

#### Barcode separation

We start the analysis with the FLASH2-merged amplicons.
First we select all merged amplicons that start and end with the theoretical sequence, and then separate all reads with the given barcode combinations letting no deviations from the barcode sequences. 

In [ ]:
fwd = ['gctatg', 'ctcagt', 'agcgta', 'gagtca', 'tctgac', 'aagctg', 'gtccat', 'cgatgt']
fwd = list(map(lambda x: x.upper(), fwd))
rev = fwd
locus_sides = re.compile(r'GTCCGCCATC[ACTG]*GGCGGCCCGG')
locus_sides_rc = re.compile(r'CCGGGCCGCC[ACGT]*GATGGCGGAC')
 
outfile = open("locus1.all.fa", "w")

cc = 0
with gzip.open("merged/locus1.extendedFrags.fastq.gz", "rt") as f:
    for record in SeqIO.parse(f, "fastq"):
        if xpc_sides.search(str(record.seq)):
            outfile.write(">" + record.id + "\n")
            outfile.write(str(record.seq) + "\n")
        if xpc_sides_rc.search(str(record.seq)):
            outfile.write(">" + record.id + "\n")
            outfile.write(str(record.seq.reverse_complement()) + "\n")


outfile.close()

In [ ]:
fwd = ['gctatg', 'ctcagt', 'agcgta', 'gagtca', 'tctgac', 'aagctg', 'gtccat', 'cgatgt']
fwd = list(map(lambda x: x.upper(), fwd))
rev = fwd


fnames  = list(map(lambda x: "F" + str(x), range(1, 9)))
rnames  = list(map(lambda x: "R" + str(x), range(1, 9)))
d       = dict()
counter = dict()

num_else = 0
else_seq = []

mat = np.zeros((8, 8))

for i in ((x + "_" + y) for x in fnames for y in rnames):
    d[i]       = []
    counter[i] = 0


for record in SeqIO.parse("locus1.all.fa", "fasta"):
    detected_f = record.seq[3:9]
    detected_r = record.seq[len(record.seq)-9:len(record.seq)-3]
    detected_r = detected_r.reverse_complement()
    try:
        address = "F" + str(fwd.index(detected_f) + 1) + "_R" + str(rev.index(detected_r)+1)
        d[address].append(record)
        counter[address] += 1
        mat[fwd.index(detected_f), rev.index(detected_r)] += 1                                                            
    except ValueError:
        num_else += 1
        else_seq.append(detected_f + ":" + detected_r)

if not os.path.exists("locus1_sep"):
    os.makedirs("locus1_sep")

for sample in d:
    for amplicon in d[sample]:
        with open(str("locus1_sep/" + sample + ".fasta"), "a") as f:
            f.write(">" + str(amplicon.id) + "\n")
            f.write(str(amplicon.seq) + "\n")


Heatmap of sequences with each barcode combination. We can see that the used combinations yield around 200.000 sequences, with a general background noise of ~5.000-10.000 due to barcode cross-contaminations.

In [ ]:
plt.figure(figsize = (7, 7))
im = plt.imshow(mat)
plt.xticks(range(len(fnames)), fnames)
plt.yticks(range(len(rnames)), rnames)
plt.title("BLNK primer combinations")
for i in range(len(fnames)):
    for j in range(len(rnames)):
        text = plt.text(j, i, mat[i, j],
                       ha="center", va="center", color="black")

#### Alignment

Global alignment of each amplicon to the theoretical locus1 sequence with **needle**. 

In [ ]:
import subprocess
import time
import os

needle = "/Users/potiadam/programok/EMBOSS-6.6.0/emboss/needle"
ref = "/Users/potiadam/Documents/CRISPR_amplicon/ref/locus1.fa"


sttime = time.time()

good_samples = ['F1_R2', 'F2_R3', 'F3_R4', 'F4_R5', 'F5_R6', 'F6_R7', 'F7_R8', 'F8_R4', 'F7_R5', 'F5_R7', 'F8_R4']
genotypes = ['BRCA1', 'BRCA2', 'RAD52', 'RAD54', 'RAD51C', 'XRCC2', 'XRCC3', 'CHK2', 'ATM',  'WT', 'PALB2']

if not os.path.exists("locus1_needle"):
    os.mkdir("locus1_needle")

for i in good_samples:
    amplicon = "/Users/potiadam/Documents/CRISPR_amplicon/locus1_sep/" + i + ".fasta"
    outfile = "/Users/potiadam/Documents/CRISPR_amplicon/locus1_needle/" + i + ".go25.sep.needle"
    
    cmd0 = "awk 'NR % 2 == 1 {print $0} NR % 2 == 0 {print substr($1, 10, length($1)-18)}' " + amplicon + " > test.fa"
    subprocess.call(cmd0, shell = True)
    cmd = needle + " -asequence " + ref + " -bsequence test.fa -outfile " + outfile
    cmd = cmd + " -gapopen 25 -gapextend 0.5 -awidth3 500"
    print(cmd)
    subprocess.call(cmd, shell = True)
    
print("It all took " + str(time.time() - sttime), " seconds")

#### Indel analysis

All used functions are in a separate scripts. Each fasta file is read into memory and deletions are detected.

In [ ]:
from CRISPR_analysis_tools import *

good_samples = ['F1_R2', 'F2_R3', 'F3_R4', 'F4_R5', 'F5_R6', 'F6_R7', 'F7_R8', 'F8_R4', 'F7_R5', 'F5_R7', 'F8_R4']
genotypes = ['BRCA1', 'BRCA2', 'RAD52', 'RAD54', 'RAD51C', 'XRCC2', 'XRCC3', 'CHK2', 'ATM',  'WT', 'PALB2']

path = r"locus1_needle/"
all_data = []
for i in good_samples:
    all_data.append(initiate(path+i+".go25.sep.needle",96,400))


A stackplot is generated for each genotype, which will show for each genotype a characteristic deletion pattern according to the starts and length of all events.

In [ ]:
realseq = 'GTCCGCCATCTTTCAAACCGCCTCCCGCCCTCTCTCGCGAGACTTTATGGCCCGTCCTCTTCCCCTCCGAGGGGGCGGGATCTGCTCGCCGCTATGGCGAGGAAGCGCAAAGCGTCTGTCCCGCGGGCGTCTGCCGGCAAAAGGCGGCCCGG'

fig, axs = plt.subplots(5,3,figsize = (12,10),sharex = True)
axs = axs.ravel()
xvals = range(1,len(realseq))
plt.subplots_adjust(left=0.05, bottom=0.06, right=0.98, top=0.95, wspace=0.18, hspace=0.2)
for d in range(len(all_data)):
    read_num = len(all_data[d][0])
    mvals = [0 for c in range(1,len(realseq))]
    for m in all_data[d][3]:
        index = realseq.find(m.del_seq,m.startpos,m.endpos)
        inslen = len(m.ins_seq)
        for c in range(index,index+len(m.del_seq)):
            mvals[c] += 1
    
    yvals = [0 for c in range(1,len(realseq))]
    for c in all_data[d][1]:
        for i in range(c.startpos,c.endpos):
            yvals[i] += 1
    
    for i in range(len(yvals)):
        yvals[i] = yvals[i] / read_num *100
        mvals[i] = mvals[i] / read_num *100
    
    axs[d].stackplot(xvals,mvals,yvals)
    axs[d].axvline(96,0,1,color = 'black')
    axs[d].set_title(genotypes[d], fontsize = 14, fontweight = 'bold')
    axs[d].tick_params(labelsize = 13)


axs[1].legend(['Cut site','Deletion with insertion','Deletion only'], fontsize = 14,loc = (-0.9,0.2))
fig.text(0.5, 0.0, 'Relative position [bp]', ha='center', va='center',fontsize = 16)
fig.text(0.0, 0.5,  'Ratio among all reads [%]', ha='center', va='center', rotation='vertical',fontsize = 16)
plt.xlim(xmin = 40,xmax = 140)
plt.show()